# Convolutional Layer

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import os
import cv2

In [2]:
import tensorflow as tf

In [164]:
from convolution2d import Convolution as cv

### CL Class

In [202]:
path = '/Users/rajeshr/Desktop/Group_22/selected/'

images = []
label = []
for i in os.listdir(path):
    image = cv2.imread(path+i, cv2.IMREAD_GRAYSCALE)
    try:
        images.append(cv2.resize(image, (224, 224)))
        label.append(i)
    except:
        pass
    
images = np.array(images)
label = np.array(label)

In [171]:
class layer():
    
    def __init__(self):
        pass
    
    def kaiming(self, n, size):
        mean, sd = 0, np.sqrt(2/n)
        weights = np.random.normal(mean, sd, size=size)
        return weights
    
    def ReLUActivation(x):
        out = []
        for i in x:
            for j in i:
                if j>=0:
                    out.append(j)
                else:
                    out.append(0)
        return np.array(out).reshape(x.shape)
    
    def ConVlayer(self, image, K, size, n):
        #n = 1
        output = []
        ReLUout = []
        for i in range(K):
            filtr = self.kaiming(n, size)
            res = cv.convolve(cv, inpt=image, filtr=filtr, stride = 1, padding = 0)
            output.append(res)
            ReLUout.append(ReLUActivation(res))
            
        self.output = np.array(output)
        self.ReLUout = np.array(ReLUout)
        
    def ConVDepthlayer(self, images, K, size, n):
        output = []
        ReLUout = []
        for i in range(K):
            filtr = self.kaiming(n, size)
            res = cv.convwithDepth(cv, inpt=images, filtr=filtr, stride = 1, padding = 0)
            output.append(res)
            ReLUout.append(ReLUActivation(res))
        
        self.output = np.array(output)
        self.ReLUout = np.array(ReLUout)

In [207]:
image = images[0]    
x, y = filtr.shape
n = x*y

layer1 = layer()
layer1.ConVlayer(image, 32, (3, 3), n)

In [208]:
for image in layer1.ReLUout:
    cv2.imwrite('/Users/rajeshr/Desktop/feature_map'+str(np.random.randint(1000))+'.jpg', image)

In [209]:
layer2 = layer()
layer2.ConVDepthlayer(layer1.ReLUout, 64, (3, 3), n)

In [210]:
for image in layer2.ReLUout:
    cv2.imwrite('/Users/rajeshr/Desktop/feature_map'+str(np.random.randint(1000))+'.jpg', image)